In [ ]:
%matplotlib inline
import nivapy3 as nivapy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import critical_loads as cl
import statsmodels.formula.api as sm

plt.style.use('ggplot')

In [ ]:
# Connect to Docker PostGIS
eng = nivapy.da.connect(src='postgres',
                        db_name='critical_loads',
                        port = 25432)

# Critical loads for soil (high-resolution method; 2018 onwards)